In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
import matplotlib.path as mpath
import pandas as pd
import json

In [2]:
import sys
sys.path.append("../")
from ccfm.ccfm import write_cfm_tri_meshes

In [3]:
!gmt grd2xyz ../misc_data/cascadia_interface/cas_slab2_dep_02.24.18.grd > ../misc_data/cascadia_interface/cas_slab2_dep_02.24.18.xyz

In [4]:
slab2_pts_orig = pd.read_csv("../input_fault_data/slab2/cas_slab2_dep_02.24.18.xyz",
                             sep="\t", names=['lon', 'lat', 'depth'])

slab2_pts_orig.tail()

,lon,lat,depth
137536,244.80,35.0,NaN
137537,244.85,35.0,NaN
137538,244.90,35.0,NaN
137539,244.95,35.0,NaN
137540,245.00,35.0,NaN


In [5]:
slab2_pts_orig.lon -= 360.0
slab2_pts_orig.depth *= 1000.0

In [6]:
slab2_dropna = slab2_pts_orig.dropna().reset_index()

In [7]:
slab2_dropna.to_csv("../input_fault_data/slab2/cas_slab2_dep_02.24.18_nonull.csv", index=False)

In [8]:
slab2_dropna.tail()

,index,lon,lat,depth
26632,111092,-119.75,38.65,-416901.062012
26633,111450,-119.90,38.60,-393525.604248
26634,111451,-119.85,38.60,-402742.065430
26635,111452,-119.80,38.60,-412147.552490
26636,111813,-119.80,38.55,-416709.960938


In [9]:
triang = mtri.Triangulation(slab2_dropna.lon.values, slab2_dropna.lat.values)

In [10]:
def _get_tri_pts(tri, pts):
    p1 = pts.loc[tri[0]]
    p2 = pts.loc[tri[1]]
    p3 = pts.loc[tri[2]]
    
    return [p1, p2, p3]

def tri_centroid(tri, pts=slab2_dropna):
    p1, p2, p3 = _get_tri_pts(tri, pts)
    
    lon = np.mean([p1.lon, p2.lon, p3.lon])
    lat = np.mean([p1.lat, p2.lat, p3.lat])
    depth = np.mean([p1.depth, p2.depth, p3.depth])
    
    return lon, lat, depth

In [11]:
tri_centroid(triang.triangles[0])

(-118.85000000000001, 47.96666666666666, -334596.8322756667)

In [12]:
tri_centroids = [tri_centroid(tri) for tri in triang.triangles]

In [13]:
centroid_latlons = np.array([[t[0],t[1]] for t in tri_centroids])

In [14]:
# this file was made in QGIS by manually making a polygon 
# connecting all of the exterior points of the slab2_dropna points

bound_file = "../input_fault_data/slab2/cas_slab2_dep_02.24.18_nonull_bounds.geojson"

with open(bound_file) as f:
    pbj = json.load(f)

In [15]:
bounds = pbj['features'][0]['geometry']['coordinates'][0]

bound_path = mpath.Path(bounds)

In [16]:
pts_in_bounds = bound_path.contains_points(centroid_latlons)

In [17]:
def tri_to_feature(tri, pts=slab2_dropna):
    p1, p2, p3 = _get_tri_pts(tri, pts)
    f = {'geometry': {
            'coordinates': [
                [[np.round(p1.lon, 5), np.round(p1.lat, 5), np.round(p1.depth, 1)],
                 [np.round(p2.lon, 5), np.round(p2.lat, 5), np.round(p2.depth, 1)],
                 [np.round(p3.lon, 5), np.round(p3.lat, 5), np.round(p3.depth, 1)],
                 [np.round(p1.lon, 5), np.round(p1.lat, 5), np.round(p1.depth, 1)]]
                ],
             'type': 'Polygon'},
        'properties': {},
        'type': 'Feature'
    }
    
    return f

In [18]:
tri_to_feature(triang.triangles[0])

{'geometry': {'coordinates': [[[-118.95, 48.05, -308649.8],
    [-118.8, 47.3, -318528.5],
    [-118.8, 48.55, -376612.2],
    [-118.95, 48.05, -308649.8]]],
  'type': 'Polygon'},
 'properties': {},
 'type': 'Feature'}

In [19]:
tris_in_boundary = [tri_to_feature(tri)
                    for i, tri in enumerate(triang.triangles)
                    if pts_in_bounds[i]
                   ]

In [20]:
tri_geojson = {
    'type': 'FeatureCollection',
    'name': 'slab2_cascadia_interface',
    'crs': {'type': 'name',
    'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
    'features': tris_in_boundary
}

In [21]:
with open(
    "../crescent_cfm_files/slab2_cascadia_interface.geojson", 'w') as f:
    json.dump(tri_geojson, f)

In [22]:
def tri_to_part(tri, pts=slab2_dropna):
    p1, p2, p3 = _get_tri_pts(tri, pts)
    coords = [[np.round(p1.lon, 5), np.round(p1.lat, 5), np.round(p1.depth, 1)],
              [np.round(p2.lon, 5), np.round(p2.lat, 5), np.round(p2.depth, 1)],
              [np.round(p3.lon, 5), np.round(p3.lat, 5), np.round(p3.depth, 1)],
              [np.round(p1.lon, 5), np.round(p1.lat, 5), np.round(p1.depth, 1)]]
    return coords
    

In [23]:
mesh = [tri_to_part(tri)
        for i, tri in enumerate(triang.triangles)
        if pts_in_bounds[i]
       ]

In [24]:
mesh_info = {'properties':{'name': 'slab2_cascadia_interface', 'source': 'Hayes et al. 2018'}}

In [25]:
write_cfm_tri_meshes(
    "../crescent_cfm_files/slab2_cascadia_interface_multipart.geojson",
    [mesh],
    [mesh_info],
    minify=True
)